In [2]:
import pandas as pd
import numpy as np

Create a DataFrame with a column `adj` where `1` for adjectives and `0` for verbs and nouns.
All data is used.

In [3]:
adjs = pd.read_csv('../Data/FlauBERT_WE/all_adjectives_we.csv', index_col=0)
adjs['adj'] = 1

In [4]:
nouns = pd.read_csv('../Data/FlauBERT_WE/all_nouns_we.csv', index_col=0)

In [5]:
verbs = pd.read_csv('../Data/FlauBERT_WE/all_verb_we.csv', index_col=0)

In [6]:
verbs_nouns = pd.concat([verbs, nouns])
verbs_nouns['adj'] = 0

In [7]:
data = pd.concat([adjs, verbs_nouns]).drop(columns=['number', 'gender'])

In [8]:
# Shuffle
data = data.sample(frac=1)

# Normalize data

In [9]:
normalized_data = (data - data.min())/(data.max() - data.min())

# Testing the correlation of verbness-ness

Now for the top frequent nouns and verbs, we can go through all the dimensions and find the correlation of each  dimension with the PoS values: 0 for verbs and 1 for nouns.

In [11]:
pos_corr_freq_df = pd.DataFrame(columns=['corr_with_pos'])
pos_list = []

# Go through dimensions from 0 to 511
for i in range(512):
    # Find the correlation of the current column with "freqlemlivres" and add to the list
    pos_list.append(normalized_data.loc[:, [str(i), 'adj']].corr().iloc[0][-1])

# Use the list values as 
pos_corr_freq_df['corr_with_pos'] = pos_list
pos_corr_freq_df['sign'] = np.sign(pos_corr_freq_df['corr_with_pos'])
pos_corr_freq_df['abs'] = abs(pos_corr_freq_df['corr_with_pos'])
pos_corr_freq_df.sort_values('abs', ascending=False)[:10]

,corr_with_pos,sign,abs
276,-0.189093,-1.0,0.189093
478,-0.175148,-1.0,0.175148
2,0.158330,1.0,0.158330
370,-0.155414,-1.0,0.155414
158,0.139729,1.0,0.139729
220,-0.138496,-1.0,0.138496
24,0.134459,1.0,0.134459
299,-0.133828,-1.0,0.133828
50,0.133630,1.0,0.133630
301,-0.133417,-1.0,0.133417


Test for unique words:

In [10]:
word, count = np.unique(normalized_data.index, return_counts=True)
unique_words = [x[0] for x in list(filter(lambda x: x[1] == 1, zip(word, count)))]

In [11]:
unique_norm = normalized_data[normalized_data.index.isin(unique_words)]

In [12]:
pos_corr_freq_df = pd.DataFrame(columns=['corr_with_pos'])
pos_list = []

# Go through dimensions from 0 to 511
for i in range(512):
    # Find the correlation of the current column with "freqlemlivres" and add to the list
    pos_list.append(unique_norm.loc[:, [str(i), 'adj']].corr().iloc[0][-1])

# Use the list values as 
pos_corr_freq_df['corr_with_pos'] = pos_list
pos_corr_freq_df['sign'] = np.sign(pos_corr_freq_df['corr_with_pos'])
pos_corr_freq_df['abs'] = abs(pos_corr_freq_df['corr_with_pos'])
pos_corr_freq_df.sort_values('abs', ascending=False)[:10]

,corr_with_pos,sign,abs
158,0.252340,1.0,0.252340
220,-0.211063,-1.0,0.211063
478,-0.194980,-1.0,0.194980
439,0.182952,1.0,0.182952
464,0.168112,1.0,0.168112
250,-0.165360,-1.0,0.165360
50,0.160972,1.0,0.160972
47,-0.160720,-1.0,0.160720
222,-0.160035,-1.0,0.160035
119,0.156363,1.0,0.156363


In [15]:
w1 = list(pd.read_csv('../Data/Dimensions/PoS/adj.csv', index_col=0).iloc[:, 0].values)

w1.extend(pos_corr_freq_df.sort_values('abs', ascending=False)[:10].index)

pd.DataFrame(w1).to_csv('../Data/Dimensions/PoS/adj.csv')